In [ ]:
# Install necessary libraries
!pip install numpy pandas scikit-learn nltk tensorflow keras


In [ ]:
 #Import required libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Download NLTK data
import nltk
print(nltk.data.path)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.data.find('tokenizers/punkt')
nltk.data.path.append('C:/Users/your-username/nltk_data')

['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/path/to/nltk_data', '/path/to/nltk_data']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
 #Load the dataset
 from google.colab import files
uploaded = files.upload()


Saving news.csv to news.csv


In [ ]:
# Load the dataset
df = pd.read_csv('news.csv')


In [ ]:
# Display the first few rows of the dataset
print(df.head())


   Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL  


In [ ]:
#Data Cleaning and Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a string
    return ' '.join(tokens)

# Apply preprocessing to the 'text' column
df['cleaned_text'] = df['text'].apply(preprocess_text)

In [ ]:
 #Split the dataset into training and testing sets
 X = df['cleaned_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Feature Extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()


In [ ]:
#Train an LSTM Model
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [ ]:
# Pad sequences to ensure uniform input size
max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [ ]:
# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the LSTM model
# Convert y_train and y_test to numerical labels (0 and 1)
y_train = pd.get_dummies(y_train, drop_first=True)['REAL'].astype(int) # Assuming 'REAL' is the positive class
y_test = pd.get_dummies(y_test, drop_first=True)['REAL'].astype(int)  # Assuming 'REAL' is the positive class


lstm_model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 334ms/step - accuracy: 0.6323 - loss: 0.6490 - val_accuracy: 0.8205 - val_loss: 0.3913
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 35s 312ms/step - accuracy: 0.8715 - loss: 0.3296 - val_accuracy: 0.8205 - val_loss: 0.4709
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 334ms/step - accuracy: 0.9034 - loss: 0.2477 - val_accuracy: 0.8166 - val_loss: 0.3933
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 40s 313ms/step - accuracy: 0.9222 - loss: 0.2058 - val_accuracy: 0.8462 - val_loss: 0.3991
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 307ms/step - accuracy: 0.9478 - loss: 0.1490 - val_accuracy: 0.8491 - val_loss: 0.4673


In [ ]:
# Evaluate LSTM Model
y_pred_lstm = (lstm_model.predict(X_test_pad) > 0.5).astype(int)
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))
print("Classification Report:\n", classification_report(y_test, y_pred_lstm))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lstm))

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 136ms/step
LSTM Accuracy: 0.861878453038674
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.90      0.87       628
           1       0.89      0.82      0.86       639

    accuracy                           0.86      1267
   macro avg       0.86      0.86      0.86      1267
weighted avg       0.86      0.86      0.86      1267

Confusion Matrix:
 [[566  62]
 [113 526]]


In [ ]:

import joblib
lstm_model.save('lstm_model.h5')

In [ ]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [ ]:
from google.colab import files

files.download('lstm_model.h5')
files.download('tfidf_vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')